In [1]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

import numpy as np
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split 
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

from os import walk
import cv2

from skimage import feature
from sklearn.model_selection import GridSearchCV #hyper param tunning
from sklearn.svm import SVC #predict algorithm
from sklearn.metrics import classification_report #evaluation

In [2]:
def loadData(dirPath):
    X=[]
    y=[]
    for (dirpath, dirnames, filenames) in walk(dirPath):
        for filename in filenames:
            if filename != '.DS_Store': 
                img = cv2.cvtColor(cv2.imread(dirPath+'/'+filename), cv2.COLOR_BGR2GRAY)
                img = np.array(img, dtype='float64').flatten()
                label = int(filename.split('_')[0])
                X.append(img)
                y.append(label)
    return X, y

def predict_image(X, model):
    y_pred = []
    for i in X: 
        label = model.predict(i)
        y_pred.append(label[0])
    return y_pred

def accuracy(y, y_pred):
    return accuracy_score(y, y_pred)

In [3]:
X_nir, y_nir = loadData('dataset2/NIR')
X_vsb, y_vsb = loadData('dataset2/VSB')

X_train_nir, X_test_nir, y_train_nir, y_test_nir = train_test_split(X_nir, y_nir, test_size=0.2, random_state=42)
X_train_vsb, X_test_vsb, y_train_vsb, y_test_vsb = train_test_split(X_vsb, y_vsb, test_size=0.2, random_state=42)

In [4]:
# public static LBPHFaceRecognizer create(int radius, int neighbors, int grid_x, int grid_y, double threshold)

nir_model =  PCA(n_components=20, svd_solver='randomized',
          whiten=True).fit(X_train_nir)
X_train_nir = nir_model.transform(X_train_nir)
X_test_nir = nir_model.transform(X_test_nir)
param_grid = {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
              'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1], }
clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced'), param_grid)
clf = clf.fit(X_train_nir, y_train_nir)


y_pred = clf.predict(X_test_nir)
print("NIR Accuracy: "+ str(accuracy(y_test_nir, y_pred)))


vsb_model =  PCA(n_components=20, svd_solver='randomized',
          whiten=True).fit(X_train_vsb)
X_train_vsb = vsb_model.transform(X_train_vsb)
X_test_vsb = vsb_model.transform(X_test_vsb)
#vsb_model.train(X_train_vsb, np.array(y_train_vsb))
param_grid = {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
              'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1], }
clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced'), param_grid)
clf = clf.fit(X_train_vsb, y_train_vsb)


y_pred = clf.predict(X_test_vsb)
print("VSB Accuracy: "+ str(accuracy(y_test_vsb, y_pred)))

NIR Accuracy: 0.92
VSB Accuracy: 0.86
